In [6]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data 
from torch.autograd import Variable

In [7]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t',header=None)
training_set.head() #usuario/movie id/movie score/time(irrelevant)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [8]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)
training_set = np.array(training_set, dtype='int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)
test_set = np.array(test_set, dtype='int')

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [9]:
#Convertir los datos al formato de PyTorch
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [10]:
#Pasar el dataset a Tensores de PyTorch
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [11]:
#Binarizar
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [12]:
#Modelo
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0-vk), 0)
    self.a += torch.sum((ph0-phk), 0)


In [13]:
#Hyperparametros
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [14]:
#Entrenamiento
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print(f'Epoca: {epoch} \t Perdida: {str((train_loss/s).numpy())} ')
   

Epoca: 1 	 Perdida: 0.3464993 
Epoca: 2 	 Perdida: 0.23835018 
Epoca: 3 	 Perdida: 0.25260508 
Epoca: 4 	 Perdida: 0.249633 
Epoca: 5 	 Perdida: 0.25102055 
Epoca: 6 	 Perdida: 0.24669974 
Epoca: 7 	 Perdida: 0.24910516 
Epoca: 8 	 Perdida: 0.24620755 
Epoca: 9 	 Perdida: 0.2500258 
Epoca: 10 	 Perdida: 0.24739599 


In [15]:
#Prueba
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    s += 1.
print('Perdida en el dataset de prueba: '+str((test_loss/s).numpy()))

Perdida en el dataset de prueba: 0.25920084
